In [1]:
import os
from os import listdir, mkdir
from os.path import isdir, join
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import histogram
from skimage import exposure

## Generate Bash Commands to Perform Registration 

In [ ]:
# Write commands for elastix test set registration
# Add parameter folder name and paramter file names (must be saved in ./parameter-files folder)
which_set = 'Test_Set' # folder name where results will be saved
param_1 = './parameter-files/par0009.affine-hist256.txt'
param_2 = './parameter-files/par0009.bspline-hist256.txt'
data_dir = f"./pre-processed-data/matched-to-mni/{which_set}/"
labels_dir = f"./data/{which_set}/"
test_ims = [f for f in listdir(data_dir) if f.endswith(".nii.gz")]
result_folder = f"./registered-data/Par0009-matched-to-mni-256hist/{which_set}/"
mkdir(result_folder)
mkdir(os.path.join(result_folder,"templates"))
mkdir(os.path.join(result_folder,"labels"))

In [ ]:
# Section writes to file in src folder
fixed_im_path = './data/MNITemplateAtlas-norm/masked_template.nii.gz'

with open(f"elastix_{which_set}", 'w+') as out_f:
    
    for chosen_im in test_ims:
        moving_im_path = os.path.join(data_dir, chosen_im)
        result_path = os.path.join(result_folder, "templates", chosen_im[:-7])
        param_1_path = param_1
        param_2_path = param_2    
        mkdir(result_path)
        cmd = f"elastix -f {fixed_im_path} -m {moving_im_path} -out {result_path} -p {param_1_path} -p {param_2_path}\n"
        out_f.write(cmd)

In [ ]:
with open(f"transformix_{which_set}", 'w+') as out_f:
    for chosen_im in test_ims:
        labels_path = os.path.join(labels_dir, f'{chosen_im[:-7]}/{chosen_im[:-7]}_seg.nii.gz')
        result_path = os.path.join(result_folder, f"labels/{chosen_im[:-7]}")
        mkdir(result_path)
        parameters_path = os.path.join(result_folder, f"templates/{chosen_im[:-7]}/TransformParameters.1.txt")
        cmd = f"transformix -in {labels_path} -out {result_path} -tp {parameters_path}\n"
        out_f.write(cmd)

In [ ]:
result_folder = "./registered-data/Training_Set/"
#Change line "(FinalBSplineInterpolationOrder 3)" in Transform parameters files
bad_files = []

for chosen_im in test_ims:
    try:
        parameters0_path = os.path.join(result_folder, "templates", f"{chosen_im}/TransformParameters.0.txt")
        parameters1_path = os.path.join(result_folder, "templates", f"{chosen_im}/TransformParameters.1.txt")
        
        replace(parameters0_path,"(FinalBSplineInterpolationOrder 3)","(FinalBSplineInterpolationOrder 0)")
        #replace(parameters0_path, '(ResultImagePixelType "short")', '(ResultImagePixelType "float")')
        #replace(parameters0_path, '(ResultImageFormat "nii")', '(ResultImageFormat "nii.gz")')
        
        replace(parameters1_path,"(FinalBSplineInterpolationOrder 3)","(FinalBSplineInterpolationOrder 0)")
        #replace(parameters1_path, '(ResultImagePixelType "short")', '(ResultImagePixelType "float")')
        #replace(parameters1_path, '(ResultImageFormat "nii")', '(ResultImageFormat "nii.gz")')
        
    except IOError as e:
        print(e)
        bad_files.append(f)

print(bad_files)